In [2]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [51]:
openai.organization = os.getenv("CHATGPTORG")
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x142543360> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

## Check if the connection and saving is working


In [52]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7EIKyqdR1tZmczrknRL0RukIJoO7W at 0x142a48810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played between the Los Angeles Dodgers and the Tampa Bay Rays, and it was held at Globe Life Field in Arlington, Texas.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683641436,
  "id": "chatcmpl-7EIKyqdR1tZmczrknRL0RukIJoO7W",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 32,
    "prompt_tokens": 57,
    "total_tokens": 89
  }
}

## Start testing

In [3]:
# open our test dataframe from the csv
df = pd.read_csv('data/wdcproducts80cc20rnd000un_valid_small_with_predictions.csv')
df = pd.read_csv("data/test_datasets/curated_benchmark.csv")
df.head()

ParserError: Error tokenizing data. C error: Expected 3 fields in line 3, saw 6


In [54]:
df.shape

(4500, 24)

In [55]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [56]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "Do the following two product decriptions match. Answer with yes or no."},
        {"role": "user",
            "content": f" product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=2,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/4500 [00:00<?, ?it/s]

Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3aaf06bc46b336a41c057f5ac1d99271 in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 41b70a47b44db37339e4f5cfae57f007 in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a399c3f8fd463c5f57bd166d1ca01668 in your message.)
Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 007614b61b5df5b2834eace63af2faf0 in you

In [57]:
df.head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
0,0,36911227,NaN,Corsair Carbide 275R Tempered Glass Blanca,NaN,89.9,EUR,NaN,717069,95352771,...,36911227#95352771,0,False,"[3.8274521828, -4.2882475853]",0,[0.0],0,No.,"{'id': 'chatcmpl-7EIL15hHtAtSUZk5HX4wwtSMEjnxS', 'object': 'chat.completion', 'created': 1683641439, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 65, 'completion_tokens': 2, 'total_tokens': 67}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Corsair Carbide 275R Tempered Glass Blanca, product 2: Logitech C310 Webcam - 1 Megapixel - USB 2.0'}]"
1,1,9216392,Shimano,XT M8000 11-Speed Cassette 11-40T,"##Shimano XT M8000 11-Speed Cassette 11-40TFor consisent shifting performance out on the trails, choose the Shimano XT M8000 11-Speed Cassette in this 11-40 tooth configuration.Compatible with Shimano 11-speed mountain bike groupsets, this relatively narrow ranging cassette enables the rider to ...",79,GBP,NaN,328225,12748766,...,9216392#12748766,0,True,"[3.8432211876, -4.2999682426]",0,[0.0],0,No.,"{'id': 'chatcmpl-7EIL2VS9lErUebKPIP2xps5aXEl7M', 'object': 'chat.completion', 'created': 1683641440, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 60, 'completion_tokens': 2, 'total_tokens': 62}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: XT M8000 11-Speed Cassette 11-40T, product 2: Catalyst 2960 Plus-48TC-L'}]"
2,2,86974759,NaN,0000005990| Sony E 30mm F3.5 Macro Lens,0000005990| Sony E 30mm F3.5 Macro Lens,599.00,AUD,NaN,213496,3970721,...,86974759#3970721,0,True,"[3.7968792915, -4.2594017982]",0,[0.0],0,No.,"{'id': 'chatcmpl-7EIL2rG3noKPYeFSzWA5ZoTD5GqPU', 'object': 'chat.completion', 'created': 1683641440, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 76, 'completion_tokens': 2, 'total_tokens': 78}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: 0000005990| Sony E 30mm F3.5 Macro Lens, product 2: SONY FE 24-240mm F3.5-6.3 OSS Lens - SEL24240'}]"
3,3,85527916,NaN,"SanDisk SSD PLUS 480 GB Sata III 2.5 Inch Internal SSD, Up to 535 MB/s","DURABLE DESIGN: SanDisk Solid State Drives have a durable solid design which is engineered to resist vibration and shock, helping to keep your data safe over time.ULTRA FAST: This solid state drive performs up to 20x faster than a typical hard disk drive, with sequential read speeds of up to 535...",7.5E1,GBP,NaN,1037877,62156844,...,85527916#62156844,0,True,"[3.8346962929, -4.2963290215]",0,[0.0],0,No.,"{'id': 'chatcmpl-7EIL3VdiusdqOOHO7df5yHWMsUain', 'object': 'chat.completion', 'created': 1683641441, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 92, 'completion_tokens': 2, 'total_tokens': 94}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: SanDisk SSD PLUS 480 GB Sata III 2.5 Inch Internal SSD, Up to 535 MB/s, product 2: WD Blue 250GB PC SSD - SATA 6 Gb/s 2.5 Inch Solid State Drive -WDS..."
4,4,32153750,NaN,"E260, E360, E460 3.5K Return Program Toner Cartridge",CZ distribuce / 0734646064620 / E260A

In [58]:
# get the number of rows were the chatbot respionse is not empty
df[df['chatbot_response'] != ''].shape

(4500, 27)

In [59]:
df.tail()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question
4495,4495,31292975,Zebra,Zebra Black Resin Monochrome Ribbon - 1000 Images,Black Resin Monochrome Ribbon - 1000 ImagesMfr Code: 800015-101More Info,NaN,NaN,NaN,5836450,21556335,...,31292975#21556335,0,True,"[3.5426645279, -3.9851288795]",0,[0.0],0,Yes.,"{'id': 'chatcmpl-7EKVneWy4kkQ06NsV88WzgVkXI8BC', 'object': 'chat.completion', 'created': 1683649795, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 74, 'completion_tokens': 2, 'total_tokens': 76}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Zebra Black Resin Monochrome Ribbon - 1000 Images, product 2: Zebra 800033-801 Black Monochrome Ribbon for ZXP Series 3 Printers - 1000 Prints'}]"
4496,4496,15417028,NaN,"Samsung CF390 27"" 4ms HDMI Curved Monitor","Samsung CF390 27"" 4ms HDMI Curved Monitor",175.88,GBP,NaN,709380,29524431,...,15417028#29524431,0,False,"[3.8305664062, -4.2897052765]",0,[0.0],0,Yes.,"{'id': 'chatcmpl-7EKVnjaMBxghJatfBI53D9iTfXDt9', 'object': 'chat.completion', 'created': 1683649795, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 77, 'completion_tokens': 2, 'total_tokens': 79}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Samsung CF390 27"" 4ms HDMI Curved Monitor, product 2: Seagate Ironwolf Pro 8TB Nas HDD 7200rpm SATA 6Gb/s 256MB 3,5""'}]"
4497,4497,96595609,Daniel Wellington,Daniel Wellington Classic Sheffield Watch | Silver 40mm,Nothing is as sleek and refined as a supple black leather wristband attached to a slim and chic watch case. With the Classic Sheffield Men's Watch from Daniel Wellington all of your style needs will be met. [Details] Strap: Leather Strap Color: Black Case Color: Silver Dial Color: Eggshell white...,229.00,USD,NaN,5135259,33918678,...,96595609#33918678,0,True,"[-1.9202262163000001, 1.9179266690999999]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7EKVonVp55DqqvA3gEn3ccMyErSvZ', 'object': 'chat.completion', 'created': 1683649796, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 55, 'completion_tokens': 2, 'total_tokens': 57}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Daniel Wellington Classic Sheffield Watch | Silver 40mm, product 2: Relgio DANIEL WELLINGTON Classic Sheffield'}]"
4498,4498,23598269,NaN,HyperX Predator XMP 16GB [2x8GB 3333MHz DDR4 CL16 DIMM],NaN,379,PLN,NaN,499316,16693470,...,23598269#16693470,0,True,"[3.7833967209, -4.2472724915]",0,[0.0],0,No.,"{'id': 'chatcmpl-7EKVp3kV0196Ce7onkDmAUWvz3yAA', 'object': 'chat.completion', 'created': 1683649797, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 79, 'completion_tokens': 2, 'total_tokens': 81}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: HyperX Predator XMP 16GB [2x8GB 3333MHz DDR4 CL16 DIMM], product 2: HyperX 32GB (2x16GB) 3000MHz CL15 Predator Black'}]"
4499,4499,35690339,NaN,Sony 24-240mm f/3.5-6.3 Zoom Lens,"This Sonyzoomlens is perfect for all of the obstacles of youreverydaylife, with a range wide enough to capture all from landscapes, t

In [60]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [61]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response'].apply(lambda x: clean_response(x))

In [62]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/chat_gpt/{now}_simple_promt_all.json')

In [63]:
# look at the wrong answers
df[df['chatbot_response_clean'] != df['label']].head()

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,...,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
7,7,66462363,Audio Technica,Audio Technica ATH-M50x Professional Monitor Headphones,"This is the most critically acclaimed model in the M-Series line, praised by top audio engineers and pro audio reviewers year after year.",122.00,GBP,NaN,1889634,91241309,...,0,True,"[-1.9811097383, 1.9827538729]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7EIL572ssrcfu7OxY58ZMpeP5Ttnr', 'object': 'chat.completion', 'created': 1683641443, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 63, 'completion_tokens': 2, 'total_tokens': 65}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Audio Technica ATH-M50x Professional Monitor Headphones, product 2: Audio Technica ATH-M50x Professional Monitor Audiophile Headphones - White'}]",1
21,21,43410009,NaN,TISSOT SEASTAR 1000 AUTOMATIC T120.407.11.051.00,"DESCRIPTIONThe Tissot Seastar 1000 merges style and performance without compromising either. The diving inspiration shapes both the appearance and the functionality of this watch. It maintains its performance to a pressure of 30 bar (300m/1'000ft), combining underwater sports and a preference fo...","2,550.00",MYR,NaN,330408,76613072,...,1,False,"[1.0417864323, -1.1085938215]",0,[1.0],1,No.,"{'id': 'chatcmpl-7EILDqkrMB1TmWmw7JkXo2i9AU8bZ', 'object': 'chat.completion', 'created': 1683641451, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 77, 'completion_tokens': 2, 'total_tokens': 79}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: TISSOT SEASTAR 1000 AUTOMATIC T120.407.11.051.00, product 2: Orologio Tissot Seamaster Powermatic 80 - T1204071105100'}]",0
27,27,42324571,NaN,"3M - Privacy filter 23\"" widescreen","3M Privacy filter 23\"" widescreen (PF230W9F) - Type: Andet tilbehør",1849.48,DKK,NaN,654362,32709637,...,0,True,"[-3.4588589668, 3.627374649]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7EILHKlssAD13jxt6z7315rvpzkUK', 'object': 'chat.completion', 'created': 1683641455, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 69, 'completion_tokens': 2, 'total_tokens': 71}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: 3M - Privacy filter 23\"" widescreen, product 2: 3M MOBILE INTERACTIVE SOLUTION PF220W9F LIGHTWEIGHT FRAMED PRIVACY FILTER'}]",1
35,35,42121103,NaN,T1204171704100 Tissot Seastar 1000 Quartz,"The Tissot Seastar 1000 merges style and performance without compromising either. The diving inspiration shapes both the appearance and the functionality of this watch. It maintains its performance to a pressure of 30 bar (300m/1'000ft), combining underwater sports and a preference for a sophist...",451.0,EUR,NaN,1167881,38366111,...,0,True,"[-1.9376914501, 1.9234960079]",1,[0.0],0,Yes.,"{'id': 'chatcmpl-7EILLIEUXCGzZtS6PLAB9Swtp7pPK', 'object': 'chat.completion', 'created': 1683641459, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 72, 'completion_tokens': 2, 'total_tokens': 74}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: T12041

In [64]:
# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")

Accuracy: 0.8868888888888888


In [65]:
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

2

In [66]:
# Look at 5 complete chatbot responses dont limit the output length
# set the max_rows and max_columns options to None
# set the max_colwidth option to None
pd.set_option('display.max_colwidth', 300)

# display the first 5 chatbot responses where chatbot_response_raw is not -1
df['chatbot_response_raw'].head(3)

0    {'id': 'chatcmpl-7EIL15hHtAtSUZk5HX4wwtSMEjnxS', 'object': 'chat.completion', 'created': 1683641439, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 65, 'completion_tokens': 2, 'total_tokens': 67}, 'choices': [{'message': {
  "content": "No.",
  "role": "assistant"
}, 'finish_reason': ...
1    {'id': 'chatcmpl-7EIL2VS9lErUebKPIP2xps5aXEl7M', 'object': 'chat.completion', 'created': 1683641440, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 60, 'completion_tokens': 2, 'total_tokens': 62}, 'choices': [{'message': {
  "content": "No.",
  "role": "assistant"
}, 'finish_reason': ...
2    {'id': 'chatcmpl-7EIL2rG3noKPYeFSzWA5ZoTD5GqPU', 'object': 'chat.completion', 'created': 1683641440, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 76, 'completion_tokens': 2, 'total_tokens': 78}, 'choices': [{'message': {
  "content": "No.",
  "role": "assistant"
}, 'finish_reason': ...
Name: chatbot_response_raw, dtype: object

In [67]:
# Lets look at all mistakes only show the title_left, title_right, label and chatbot_response_clean
df[df['chatbot_response_clean'] != df['label']][['title_left', 'title_right', 'label', 'chatbot_response_clean']]


,title_left,title_right,label,chatbot_response_clean
7,Audio Technica ATH-M50x Professional Monitor Headphones,Audio Technica ATH-M50x Professional Monitor Audiophile Headphones - White,0,1
21,TISSOT SEASTAR 1000 AUTOMATIC T120.407.11.051.00,Orologio Tissot Seamaster Powermatic 80 - T1204071105100,1,0
27,"3M - Privacy filter 23\"" widescreen",3M MOBILE INTERACTIVE SOLUTION PF220W9F LIGHTWEIGHT FRAMED PRIVACY FILTER,0,1
35,T1204171704100 Tissot Seastar 1000 Quartz,Tissot Seastar 1000 T120.417.11.051.00,0,1
44,Access Point TP-Link N300 WIFI Ceiling Mount - Suporte POE,TP-Link EAP115 Wi-Fi 300Mbps Tavan Tipi Access P.,0,1
...,...,...,...,...
4482,"3M - PRIVACY FILTER LCD 27\"" 16:9",Seagate IronWolf ST10000VN0008hard drive - 10 TB - SATA 6Gb/s,0,1
4492,TAG Heuer Automatic Watch WAZ2011.BA0842,TAG Heuer Carrera Calibre 16 Automatic Men's Watch,0,1
4495,Zebra Black Resin Monochrome Ribbon - 1000 Images,Zebra 800033-801 Black Monochrome Ribbon for ZXP Series 3 Printers - 1000 Prints,0,1
4496,"Samsung CF390 27"" 4ms HDMI Curved Monitor","Seagate Ironwolf Pro 8TB Nas HDD 7200rpm SATA 6Gb/s 256MB 3,5""",0,1


In [68]:
# how many currencies are there in the dataset
df['priceCurrency_left'].unique()

array(['EUR', 'GBP', 'AUD', 'CZK', 'USD', nan, 'NOK', 'SEK', 'DKK', 'MYR',
       'CAD', 'RUB', 'SGD', 'PLN', 'NZD', 'VND', 'GHS', 'RON', 'ZAR',
       'PHP', 'AED', 'COP', 'BAM', 'NPR', 'BRL', 'EGP', 'NGN', 'UAH',
       'SAR', 'INR', 'KYD', 'TRY', 'лв.', 'HRK', 'LKR', 'HKD', 'MUR',
       'ILS', 'CHF', 'CLP', 'HUF', 'KES', 'MXN', 'BGN', 'BDT', 'eur',
       'ISK'], dtype=object)

In [69]:
# How often does priceCurrency_left and priceCurrency_right match
df[df['priceCurrency_left'] == df['priceCurrency_right']].shape[0]

753